In [3]:
%%file level-core.lp

#const width=10.

param("width",width).

dim(0..width-1).

tile((X,Y)) :- dim(X); dim(Y).
    
adj((X1,Y1),(X2,Y2)) :- 
    tile((X1,Y1));
    tile((X2,Y2));
    |X1-X2| + |Y1-Y2| == 1.
    
start((0,0)).
finish((width-1,width-1)).

% tile at most one named sprite
0 { sprite(T,(wall;gem;altar;trap)) } 1 :- tile(T).
   
% there is exactly one gem and one altar in the whole level
:- not 1 { sprite(T,altar) } 1.
:- not 1 { sprite(T,gem) } 1.
    
% there are between 2 and 5 traps in the level
:- not 2 { sprite(T,trap) } 5.
    
% the start and finish are clear
:- start(T); sprite(T,Name).
:- finish(T); sprite(T,Name).
    
#show param/2.
#show tile/1.
#show sprite/2.

Overwriting level-core.lp


In [4]:
%%file level-style.lp

% style: at least half of the map has wall sprites
:- not (width*width)/2 { sprite(T,wall) }.
 
% style: every wall has at least two neighboring walls
:- sprite(T1,wall); not 2 { sprite(T2,wall) : adj(T1,T2) }.
   
% style: altars have no surrounding sprites for two steps
:- sprite(T1,altar); not 0 { sprite(T2,S) : adj(T1,T2) } 0.
:- sprite(T1,altar); not 0 { sprite(T3,S) : adj(T1,T2), adj(T2,T3), T1 != T3 } 0.
    
% style: altars have four adjacent tiles (not on a map border)
:- sprite(T1,altar); not 4 { adj(T1,T2) }.
    
% style: gems have exactly three surrounding walls
:- sprite(T1,gem); not 3 { sprite(T2,wall) : adj(T1,T2) } 3.

Overwriting level-style.lp


In [18]:
%%file level-sim.lp

% states:
% 1 --> initial
% 2 --> after picking up gem
% 3 --> after putting gem in altar

% you start in state 1

touch(T,1) :- start(T).
    
special(T) :- sprite(T,gem).
special(T) :- sprite(T,altar).

% possible navigation paths
{ step(T1,1,T2,2):adj(T1,T2) } 1 :- touch(T1,1); sprite(T1,gem).
{ step(T1,2,T2,3):adj(T1,T2) } 1 :- touch(T1,2); sprite(T1,altar).
{ step(T1,S,T2,S):adj(T1,T2) } 1 :- touch(T1,S); not special(T1); not finish(T1).

touch(T2,S2) :- step(T1,S1,T2,S2).

% you can't touch a wall in any state
:- sprite(T,wall); touch(T,S).
    
% you can only touch a trap after picking up the gem
:- sprite(T,trap); touch(T,S); S != 2.

% the finish tile must be touched in state 3
completed :- finish(T); touch(T,3).
:- not completed.
    
#show touch/2.

Overwriting level-sim.lp


In [19]:
%%file level-shortcuts.lp

__level_design(sprite(T,Name)) :- sprite(T,Name).
    
wander :- width*width/2 { touch(T,S) }.
trap_step :- touch(T,S); sprite(T,trap).
a_trap_went_untouched :- sprite(T,trap); not touch(T,2).
    
__concept :- wander; trap_step; not a_trap_went_untouched.
        
        
#show __level_design/1.
#show __concept/0.

Overwriting level-shortcuts.lp


In [10]:
!./clingo-4.5.0-macos-10.9/clingo level-core.lp

clingo version 4.5.0
Reading from level-core.lp
Solving...
Answer: 1
param("width",10) tile((0,0)) tile((1,0)) tile((2,0)) tile((3,0)) tile((4,0)) tile((5,0)) tile((6,0)) tile((7,0)) tile((8,0)) tile((9,0)) tile((0,1)) tile((1,1)) tile((2,1)) tile((3,1)) tile((4,1)) tile((5,1)) tile((6,1)) tile((7,1)) tile((8,1)) tile((9,1)) tile((0,2)) tile((1,2)) tile((2,2)) tile((3,2)) tile((4,2)) tile((5,2)) tile((6,2)) tile((7,2)) tile((8,2)) tile((9,2)) tile((0,3)) tile((1,3)) tile((2,3)) tile((3,3)) tile((4,3)) tile((5,3)) tile((6,3)) tile((7,3)) tile((8,3)) tile((9,3)) tile((0,4)) tile((1,4)) tile((2,4)) tile((3,4)) tile((4,4)) tile((5,4)) tile((6,4)) tile((7,4)) tile((8,4)) tile((9,4)) tile((0,5)) tile((1,5)) tile((2,5)) tile((3,5)) tile((4,5)) tile((5,5)) tile((6,5)) tile((7,5)) tile((8,5)) tile((9,5)) tile((0,6)) tile((1,6)) tile((2,6)) tile((3,6)) tile((4,6)) tile((5,6)) tile((6,6)) tile((7,6)) tile((8,6)) tile((9,6)) tile((0,7)) tile((1,7)) tile((2,7)) tile((3,7)) tile((4,7)) tile((5,7)) t

In [11]:
!./clingo-4.5.0-macos-10.9/clingo level-core.lp level-style.lp level-sim.lp --sign-def=3 --outf=2 > example.json

In [19]:
%%file p7_visualize.py

import json
import sys
import collections

def solve(*args):
    """Run clingo with the provided argument list and return the parsed JSON result."""
    
    CLINGO = "./clingo-4.5.0-macos-10.9/clingo"
    
    clingo = subprocess.Popen(
        [CLINGO, "--outf=2"] + list(args),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE)
    out, err = clingo.communicate()
    if err:
        print err
        
    return parse_json_result(out)

def parse_json_result(out):
    """Parse the provided JSON text and extract a dict
    representing the predicates described in the first solver result."""

    with open(out) as f:
        result = json.load(f)
    
    assert len(result['Call']) > 0
    assert len(result['Call'][0]['Witnesses']) > 0
    
    witness = result['Call'][0]['Witnesses'][0]['Value']
    
    class identitydefaultdict(collections.defaultdict):
        def __missing__(self, key):
            return key
    
    preds = collections.defaultdict(set)
    env = identitydefaultdict()
    
    for atom in witness:
        if '(' in atom:
            left = atom.index('(')
            functor = atom[:left]
            arg_string = atom[left:]
            try:
                preds[functor].add( eval(arg_string, env) )
            except TypeError:
                pass # at least we tried...
            
        else:
            preds[atom] = True
    
    return dict(preds)

def solve_randomly(*args):
    """Like solve() but uses a random sign heuristic with a random seed."""
    args = list(args) + ["--sign-def=3","--seed="+str(random.randint(0,1<<30))]
    return solve(*args) 

def render_ascii_dungeon(design):
    """Given a dict of predicates, return an ASCII-art depiction of the a dungeon."""
    
    sprite = dict(design['sprite'])
    param = dict(design['param'])
    width = param['width']
    glyph = dict(space='.', wall='W', altar='a', gem='g', trap='_')
    block = ''.join([''.join([glyph[sprite.get((r,c),'space')]+' ' for c in range(width)])+'\n' for r in range(width)])
    return block

def render_ascii_touch(design, target):
    """Given a dict of predicates, return an ASCII-art depiction where the player explored
    while in the `target` state."""
    
    touch = collections.defaultdict(lambda: '-')
    for cell, state in design['touch']:
        if state == target:
            touch[cell] = str(target)
    param = dict(design['param'])
    width = param['width']
    block = ''.join([''.join([str(touch[r,c])+' ' for c in range(width)])+'\n' for r in range(width)])
    return block

def side_by_side(*blocks):
    """Horizontally merge two ASCII-art pictures."""
    
    lines = []
    for tup in zip(*map(lambda b: b.split('\n'), blocks)):
        lines.append(' '.join(tup))
    return '\n'.join(lines)

map = parse_json_result(sys.argv[1])
print render_ascii_dungeon(map)


Overwriting p7_visualize.py


In [20]:
!python p7_visualize.py example.json

. _ _ _ . . . . . . 
W W W W g W W _ . . 
W W W W W W W . . . 
W W W W W W . . a . 
W W W W W W W . . . 
_ W W W W W W W . . 
W W W W W W W W W . 
W W W W W W W W W . 
W W W W W W W W W . 
W W W W W W W W W . 



In [21]:
%%file metaS.lp

level_design(A) :- show(A,__level_design(_)).
concept(A) :- show(A,__concept).

criteria(0,0,0,A) :- level_design(A).
criteria(0,0,0,-A) :- level_design(A).
criteria(0,0,0,A) :- concept(A).

optimize(0,0,incl).

:- concept(A), not hold(A).

Writing metaS.lp


In [3]:
!./clingo-4.5.0-macos-10.9/gringo level-core.lp level-style.lp level-sim.lp level-shortcuts.lp -c width=7 \
	| ./clingo-4.5.0-macos-10.9/reify \
	| ./clingo-4.5.0-macos-10.9/clingo - meta.lp metaD.lp metaO.lp metaS.lp \
		--parallel-mode=4 --outf=2 > example_noshortcut.json

metaD.lp:67:27-35: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:67:57-66: info: atom does not occur in any rule head:
  scc(C,A')

metaD.lp:68:27-35: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:68:57-66: info: atom does not occur in any rule head:
  scc(C,A')

metaD.lp:70:27-35: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:71:27-35: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:73:15-23: info: atom does not occur in any rule head:
  scc(#X0,#P1)

metaD.lp:73:31-39: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:75:25-33: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:78:58-70: info: atom does not occur in any rule head:
  not scc(C,L)

metaD.lp:79:62-70: info: atom does not occur in any rule head:
  scc(C,L)

metaD.lp:83:8-16: info: atom does not occur in any rule head:
  scc(C,A)

metaO.lp:71:37-50: info: atom does not occur in any rule head:
  minimize(J,S)

metaO.lp:71

In [4]:
!python p7_visualize.py example_noshortcut.json

. . . _ . . . 
_ W W . . a . 
W W W W . . . 
W W W W . . . 
W W g . . W W 
W W W W . W W 
W W W W . . . 



In [21]:
%%file p7_driver.py
import json
import sys
import collections
import random
import subprocess

def solve(*args):
    """Run clingo with the provided argument list and return the parsed JSON result."""
    
    CLINGO = "./clingo-4.5.0-macos-10.9/clingo"
    
    clingo = subprocess.Popen(
        [CLINGO, "--outf=2"] + list(args),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE)
    out, err = clingo.communicate()
    if err:
        print err
        
    return parse_json_result(out)

def parse_json_result(out):
    """Parse the provided JSON text and extract a dict
    representing the predicates described in the first solver result."""

    result = json.loads(out)
    
    assert len(result['Call']) > 0
    assert len(result['Call'][0]['Witnesses']) > 0
    
    witness = result['Call'][0]['Witnesses'][0]['Value']
    
    class identitydefaultdict(collections.defaultdict):
        def __missing__(self, key):
            return key
    
    preds = collections.defaultdict(set)
    env = identitydefaultdict()
    
    for atom in witness:
        if '(' in atom:
            left = atom.index('(')
            functor = atom[:left]
            arg_string = atom[left:]
            try:
                preds[functor].add( eval(arg_string, env) )
            except TypeError:
                pass # at least we tried...
            
        else:
            preds[atom] = True
    
    return dict(preds)

def solve_randomly(*args):
    """Like solve() but uses a random sign heuristic with a random seed."""
    args = list(args) + ["--sign-def=3","--seed="+str(random.randint(0,1<<30))]
    return solve(*args) 

def render_ascii_dungeon(design):
    """Given a dict of predicates, return an ASCII-art depiction of the a dungeon."""
    
    sprite = dict(design['sprite'])
    param = dict(design['param'])
    width = param['width']
    glyph = dict(space='.', wall='W', altar='a', gem='g', trap='_')
    block = ''.join([''.join([glyph[sprite.get((r,c),'space')]+' ' for c in range(width)])+'\n' for r in range(width)])
    return block

def render_ascii_touch(design, target):
    """Given a dict of predicates, return an ASCII-art depiction where the player explored
    while in the `target` state."""
    
    touch = collections.defaultdict(lambda: '-')
    for cell, state in design['touch']:
        if state == target:
            touch[cell] = str(target)
    param = dict(design['param'])
    width = param['width']
    block = ''.join([''.join([str(touch[r,c])+' ' for c in range(width)])+'\n' for r in range(width)])
    return block

def side_by_side(*blocks):
    """Horizontally merge two ASCII-art pictures."""
    
    lines = []
    for tup in zip(*map(lambda b: b.split('\n'), blocks)):
        lines.append(' '.join(tup))
    return '\n'.join(lines)

clingo = subprocess.Popen(
    "./gringo level-core.lp level-style.lp level-sim.lp level-shortcuts.lp -c width=7 | ./reify | ./clingo - meta.lp metaD.lp metaO.lp metaS.lp --parallel-mode=4 --outf=2",
    shell = True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE)
out, err = clingo.communicate()
if err:
    print err
    
map = parse_json_result(out)
print render_ascii_dungeon(map)

Overwriting p7_driver.py


In [26]:
!python p7_driver.py

metaD.lp:67:27-35: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:67:57-66: info: atom does not occur in any rule head:
  scc(C,A')

metaD.lp:68:27-35: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:68:57-66: info: atom does not occur in any rule head:
  scc(C,A')

metaD.lp:70:27-35: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:71:27-35: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:73:15-23: info: atom does not occur in any rule head:
  scc(#X0,#P1)

metaD.lp:73:31-39: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:75:25-33: info: atom does not occur in any rule head:
  scc(C,A)

metaD.lp:78:58-70: info: atom does not occur in any rule head:
  not scc(C,L)

metaD.lp:79:62-70: info: atom does not occur in any rule head:
  scc(C,L)

metaD.lp:83:8-16: info: atom does not occur in any rule head:
  scc(C,A)

metaO.lp:71:37-50: info: atom does not occur in any rule head:
  minimize(J,S)

metaO.lp:71